# Toxic Comment Classification Challenge
- 6가지 종류(toxic, severe_toxic, obscene, threat, identity_hate)의 악플일 확률을 예측
- 평가지표: mean column-wise ROC AUC

## imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys
import warnings
warnings.filterwarnings(action='ignore')
os.chdir('/content/drive/MyDrive/Colab Notebooks/연습하자/jigsaw-toxic-comment-classification-challenge')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/연습하자/jigsaw-toxic-comment-classification-challenge'

In [ ]:
!pip install emoji

In [ ]:
!pip uninstall torch torchtext -y
!pip install torch==2.1.0 torchtext==0.16.0

Found existing installation: torch 2.1.0
Uninstalling torch-2.1.0:
  Successfully uninstalled torch-2.1.0
Found existing installation: torchtext 0.16.0
Uninstalling torchtext-0.16.0:
  Successfully uninstalled torchtext-0.16.0
  Using cached torch-2.1.0-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchtext-0.16.0-cp311-cp311-manylinux1_x86_64.whl.metadata (7.5 kB)
Using cached torch-2.1.0-cp311-cp311-manylinux1_x86_64.whl (670.2 MB)
Using cached torchtext-0.16.0-cp311-cp311-manylinux1_x86_64.whl (2.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.1.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.1.0 which is incompatible.
torchtune 0.6.1 requires torchdata==0.11.0, but you have torchdata 0.7.0 which is incompatible.


In [ ]:
import re  # 정규표현식 처리해주는 모듈
import string
import zipfile

import emoji
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from tqdm import tqdm

In [ ]:
training_data = pd.read_csv('train.csv')
training_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
sample_data = pd.read_csv('sample_submission.csv')
sample_data.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [ ]:
test_data = pd.read_csv('test.csv')
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [ ]:
test_labels = pd.read_csv('test_labels.csv')
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


## preprocessing

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
punc = string.punctuation
punc.replace('#','')
punc.replace('!','')
punc.replace('?','')
punc = punc + "∞θ÷α•à−β∅³π‘₹´°£€\×™√²—"   # 특수문자 날리기

chat_words = {   # 채팅 용어
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

stopwords = stopwords.words('english')  # 불용어 처리

nlp = spacy.load("en_core_web_sm")   # spacy라는 라이브러리에서 전처리하는 모델을 로드

time_zone_abbreviations = [
        "UTC", "GMT", "EST", "CST", "PST", "MST",
        "EDT", "CDT", "PDT", "MDT", "CET", "EET",
        "WET", "AEST", "ACST", "AWST", "HST",
        "AKST", "IST", "JST", "KST", "NZST"
    ]

patterns = [
    r'\\[nrtbfv\\]',         # \n, \t ..etc
    '<.*?>',                 # Html tags
    r'https?://\S+|www\.\S+',# Links
    r'\ufeff',               # BOM characters
    r'^[^a-zA-Z0-9]+$',      # Non-alphanumeric tokens
    r'ｗｗｗ．\S+',            # Full-width URLs
    r'[\uf700-\uf7ff]',      # Unicode private-use chars
    r'^[－—…]+$',            # Special punctuation-only tokens
    r'[︵︶]'                # CJK parentheses
]

def preprocess(text):
    for regex in patterns:
        text = re.sub(regex, '', text)  # text에서 regex를 공백으로 대체
    text = text.translate(str.maketrans(punc, ' ' * len(punc))) # punc(문장부호)를 공백으로 대체
    text = ' '.join(word for word in text.split() if word not in time_zone_abbreviations) # 시간대 공백으로 대체
    text = ' '.join(word for word in text.split() if word not in stopwords)  # 불용어 공백으로 대체
    text = ' '.join(chat_words.get(word.lower(), word) for word in text.split())  # 줄임말 다시 늘린걸로 교체
    text = text.lower()  # 소문자로 통일
    text = emoji.demojize(text) # 이모지를 문자로 바꿔주기
    text = re.sub(r'\s+', ' ', text).strip() # 공백제거

    return text

## vocabulary construction

In [ ]:
comments = list(training_data['comment_text'])
train_iter = iter(comments)

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
  for text in data_iter:
    cleaned_text = preprocess(text)
    tokens = [ token for token in tokenizer(cleaned_text) if 1 < len(token) < 25 ]   # 길이가 25이하인 토큰만 쓰는듯..
    yield tokens

vocab = build_vocab_from_iterator(yield_tokens(train_iter),
                                  specials = ["<pad>","<unk>"],   # padding / unknown token을 특이케이스(?)로 추가
                                  max_tokens = 30002)
vocab.set_default_index(vocab["<unk>"])
PAD_IDX = vocab["<pad>"]

print(f"Final vocabulary size: {len(vocab)}")
print("Sample valid tokens:", [t for t in list(vocab.get_itos())[2:12]])

Final vocabulary size: 30002
Sample valid tokens: ['article', 'the', 'page', 'wikipedia', 'talk', 'you', 'please', 'would', 'one', 'like']


In [ ]:
torch.save(vocab,"vocab.pth")

In [ ]:
vocab

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def text_pipeline(text):
  return [
      vocab[token] if token in vocab else vocab["<unk>"]
      for token in tokenizer(preprocess(text))
  ]

def label_pipeline(labels):
  return torch.FloatTensor(labels)

### dataset preparation

In [ ]:
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import torch

class PaddedDataset(Dataset):
  def __init__(self, df, vocab, max_length=None):
    self.df = df
    self.vocab = vocab
    self.max_length = max_length
    self.label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    text = self.df.iloc[idx]['comment_text']
    labels = self.df.iloc[idx][self.label_cols].values.astype(float)

    tokens = tokenizer(preprocess(text))
    if self.max_length:
      tokens = tokens[:self.max_length]
    numericalized = [self.vocab[token] for token in tokens]

    return torch.tensor(numericalized, dtype=torch.long), torch.tensor(labels, dtype=torch.float)

In [ ]:
# 같은 배치 내의 변수의 길이를 통일하기 위한 함수 -> 가장 길이가 긴 변수의 길이에 맞게 vocab["<pad>"]로 패딩
def collate_batch(batch):
    texts, labels = zip(*batch)
    lengths = torch.tensor([len(t) for t in texts])
     # Filter invalid sequences (length <=0)
    valid_mask = lengths > 0
    if not valid_mask.all():
        texts = [t for t, valid in zip(texts, valid_mask) if valid]
        labels = [l for l, valid in zip(labels, valid_mask) if valid]
        lengths = lengths[valid_mask]

    # Add fallback for empty batch
    if len(texts) == 0:
        return torch.zeros((1,1), dtype=torch.long), torch.zeros((1,6)), torch.tensor([1])
    # Pad sequences to match longest in batch
    padded_texts = torch.nn.utils.rnn.pad_sequence(
        texts,
        batch_first=True,
        padding_value=PAD_IDX
    )

    return padded_texts, torch.stack(labels), lengths

In [ ]:
BATCH_SIZE = 64
MAX_SEQ_LEN = 256

dataset = PaddedDataset(training_data, vocab, max_length=MAX_SEQ_LEN)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch,
    pin_memory = True,
    num_workers=2
)

### model architectures

#### baseline LSTM

In [ ]:
class lstm(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, pad_idx, output_dim):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
    self.fc1 = nn.Linear(hidden_dim, output_dim)
  def forward(self, text, lengths):
    embedded = self.embedding(text)
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
    packed_output, (hidden, cell) = self.lstm(packed_embedded)
    out = self.fc1(hidden[-1])
    return torch.sigmoid(out)

#### bidirectional LSTM

In [ ]:
class BIDirectional_lstm(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, pad_idx, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = pad_idx)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(hidden_dim * 2, output_dim) # 평탄화
        self.dropout =  nn.Dropout(p = 0.3)  # 과적합 방지 -> 뉴런 제거
    def forward(self, text, lengths):
        embedded = self.embedding(text)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded,
            lengths.cpu(),
            batch_first=True,
            enforce_sorted = False
        )
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        hidden_output = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        out = self.fc1(hidden_output)
        out = self.dropout(out)
        return torch.sigmoid(out)

### training framework

In [ ]:
def train_model(model, train_loader, val_loader, epochs, learning_rate, filename):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = nn.BCELoss()
    best_val_loss = float('inf')
    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for texts, labels, lengths in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
            # Move data to device
            texts, labels = texts.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(texts, lengths)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
            optimizer.step()

            # Calculate metrics
            train_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).all(dim=1).sum().item()
            total += labels.size(0)

        # Validation Phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for texts, labels, lengths in val_loader:
                texts, labels = texts.to(device), labels.to(device)
                outputs = model(texts, lengths)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                val_correct += (predicted == labels).all(dim=1).sum().item()
                val_total += labels.size(0)

        # Epoch Statistics
        train_loss /= len(train_loader)
        train_acc = correct / total
        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        print(f"\nEpoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.2%}")
        print(f"Val Loss: {val_loss:.4f} | Acc: {val_acc:.2%}")

        # Learning rate scheduling
        scheduler.step(val_loss)

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), filename + '.pth')

    print("Training complete!")

In [ ]:
train_df, val_df = train_test_split(training_data.iloc[:, 1:], test_size=0.2)

train_dataset = PaddedDataset(train_df, vocab, max_length=256)
val_dataset = PaddedDataset(val_df, vocab, max_length=256)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collate_batch,
    pin_memory=True,
    num_workers=4
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    collate_fn=collate_batch,
    pin_memory=True,
    num_workers=4
)

In [ ]:
lstm_model = lstm(
    vocab_size=len(vocab),
    embed_dim=50,
    hidden_dim=256,
    output_dim=6,
    pad_idx=PAD_IDX
)

train_model(lstm_model, train_loader, val_loader, 5, 0.001, "lstm")

Epoch 1: 100%|██████████| 1995/1995 [02:28<00:00, 13.40it/s]



Epoch 1/5
Train Loss: 0.0975 | Acc: 90.10%
Val Loss: 0.0590 | Acc: 91.77%


Epoch 2: 100%|██████████| 1995/1995 [02:24<00:00, 13.76it/s]



Epoch 2/5
Train Loss: 0.0543 | Acc: 91.73%
Val Loss: 0.0496 | Acc: 92.16%


Epoch 3: 100%|██████████| 1995/1995 [02:22<00:00, 14.04it/s]



Epoch 3/5
Train Loss: 0.0460 | Acc: 92.16%
Val Loss: 0.0462 | Acc: 92.12%


Epoch 4: 100%|██████████| 1995/1995 [02:22<00:00, 13.97it/s]



Epoch 4/5
Train Loss: 0.0406 | Acc: 92.64%
Val Loss: 0.0451 | Acc: 92.51%


Epoch 5: 100%|██████████| 1995/1995 [02:25<00:00, 13.75it/s]



Epoch 5/5
Train Loss: 0.0360 | Acc: 93.21%
Val Loss: 0.0461 | Acc: 92.31%
Training complete!


In [ ]:
BIDirectional_model = BIDirectional_lstm(
    vocab_size=len(vocab),
    embed_dim=50,
    hidden_dim=256,
    output_dim=6,
    pad_idx=PAD_IDX
)

train_model(BIDirectional_model, train_loader, val_loader, 7, 0.0005, "bidirctional_lstm")

Epoch 1: 100%|██████████| 1995/1995 [02:34<00:00, 12.93it/s]



Epoch 1/7
Train Loss: 0.2811 | Acc: 89.56%
Val Loss: 0.0847 | Acc: 90.98%


Epoch 2: 100%|██████████| 1995/1995 [02:30<00:00, 13.23it/s]



Epoch 2/7
Train Loss: 0.2525 | Acc: 90.27%
Val Loss: 0.0678 | Acc: 91.51%


Epoch 3: 100%|██████████| 1995/1995 [02:33<00:00, 13.01it/s]


...


Epoch 5/7
Train Loss: 0.2380 | Acc: 91.15%
Val Loss: 0.0512 | Acc: 92.18%


Epoch 6: 100%|██████████| 1995/1995 [02:28<00:00, 13.44it/s]



Epoch 6/7
Train Loss: 0.2361 | Acc: 91.26%
Val Loss: 0.0494 | Acc: 92.28%


Epoch 7: 100%|██████████| 1995/1995 [02:29<00:00, 13.37it/s]



Epoch 7/7
Train Loss: 0.2324 | Acc: 91.55%
Val Loss: 0.0492 | Acc: 92.23%
Training complete!


### model evaluation

In [ ]:
ev_data = pd.concat([test_data,test_labels.iloc[:,1:]], axis=1)
# dropping -1 rows, these rows weren't used for evaluation models in the competetion and marked with -1
ev_data = ev_data[ev_data['toxic']!= -1]
ev_data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
153151,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
153154,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
153155,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [1]:
eval_dataset = PaddedDataset(ev_data, vocab, max_length=256)
eval_loader = DataLoader(
    eval_dataset,
    batch_size=512,
    collate_fn=collate_batch,
    pin_memory=True,
    num_workers=4
)

NameError: name 'PaddedDataset' is not defined

In [ ]:
def calc_roc(model):
    model.eval()
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for texts, labels, lengths in eval_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts, lengths)

            # Store batch results
            all_labels.append(labels.cpu().numpy())
            all_outputs.append(outputs.cpu().numpy())

    # Concatenate all batches
    all_labels = np.concatenate(all_labels, axis=0)
    all_outputs = np.concatenate(all_outputs, axis=0)

    # Calculate ROC-AUC for each class
    roc_scores = []
    for col in range(6):  # the original evaluation method is to take ROC-AUC scores average for the 6 classed
        if np.sum(all_labels[:, col]) > 0:
            roc = roc_auc_score(all_labels[:, col], all_outputs[:, col])
            roc_scores.append(roc)

    # Return average
    return np.mean(roc_scores)

In [ ]:
lstm_model = lstm_model = lstm(
    vocab_size=len(vocab),
    embed_dim=50,
    hidden_dim=256,
    output_dim=6,
    pad_idx=PAD_IDX
).to(device)

bi_lstm_model = BIDirectional_model = BIDirectional_lstm(
    vocab_size=len(vocab),
    embed_dim=50,
    hidden_dim=256,
    output_dim=6,
    pad_idx=PAD_IDX
).to(device)

lstm_model.load_state_dict(torch.load("lstm.pth"))
bi_lstm_model.load_state_dict(torch.load("bidirctional_lstm.pth"))

<All keys matched successfully>

In [ ]:
print("lstm model roc-auc ", calc_roc(lstm_model))
print("BiDirectional lstm model roc-auc ", calc_roc(bi_lstm_model))